<a href="https://colab.research.google.com/github/Aryan12Dubey/Bidirectional-LSTM-RNN/blob/main/Bidirectional_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake NEws Classificer USsing Bidirectional LSTM

In [ ]:
## Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv
import pandas as pd



In [ ]:
df=pd.read_csv('train.csv',engine='python',error_bad_lines=False)

Skipping line 7100: unexpected end of data


In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.isnull().sum()

id          0
title     195
author    677
text       14
label       0
dtype: int64

In [ ]:
df=df.dropna()

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
## get the independent features
X=df.drop('label',axis=1)

In [ ]:
y=df['label']

In [ ]:
## Check whether dataset is balanced or not
y.value_counts()

0    3530
1    2696
Name: label, dtype: int64

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.8.2'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [ ]:
## vocabulary size
voc_size=5000

In [ ]:
messages=X.copy()

In [ ]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[1104, 2851, 1511, 3737, 822, 938, 3909, 2068, 4743, 4581],
 [4276, 4441, 2336, 1870, 3123, 3069, 2679],
 [1935, 1900, 1035, 2591],
 [4270, 2937, 2473, 1242, 3900, 2614],
 [2852, 3123, 1691, 1292, 2225, 2726, 3123, 3858, 2328, 4204],
 [425,
  1004,
  4996,
  2534,
  552,
  1768,
  1112,
  2600,
  358,
  620,
  335,
  2605,
  1012,
  957,
  2679],
 [1042, 3415, 3127, 3661, 653, 4478, 4560, 1102, 2104, 81, 2000],
 [82, 1251, 3631, 1965, 3613, 4349, 1768, 1405, 2104, 81, 2000],
 [1824, 1115, 1987, 724, 3021, 2876, 4156, 4586, 1768, 2845],
 [4502, 3742, 1704, 4805, 932, 3371, 646, 2664],
 [707, 2552, 207, 2277, 1355, 3423, 4549, 282, 965, 4230, 634],
 [1242, 3417, 822, 2876, 1768, 3613],
 [386, 2190, 4200, 14, 3674, 4134, 1070, 4608, 845],
 [578, 3082, 3742, 3788, 2283, 1104, 3582, 2104, 81, 2000],
 [1969, 4255, 142, 267, 2694, 2104, 81, 2000],
 [2542, 2350, 798, 4526, 1338, 648, 1635, 2744, 3892, 909],
 [4084, 1599, 4441],
 [431, 1099, 704, 3505, 1768, 38, 2562, 2679],
 [4622, 2482, 2336

## Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 2068, 4743, 4581],
       [   0,    0,    0, ..., 3123, 3069, 2679],
       [   0,    0,    0, ..., 1900, 1035, 2591],
       ...,
       [   0,    0,    0, ..., 1800, 2901, 2499],
       [   0,    0,    0, ..., 4441, 2539, 1318],
       [   0,    0,    0, ..., 4016,  248, 2499]], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
131/131 [==============================] - 15s 22ms/step - loss: 0.3781 - accuracy: 0.8125 - val_loss: 0.2300 - val_accuracy: 0.8983
Epoch 2/10
131/131 [==============================] - 2s 13ms/step - loss: 0.1577 - accuracy: 0.9348 - val_loss: 0.2167 - val_accuracy: 0.9109
Epoch 3/10
131/131 [==============================] - 2s 13ms/step - loss: 0.0754 - accuracy: 0.9751 - val_loss: 0.2756 - val_accuracy: 0.9100
Epoch 4/10
131/131 [==============================] - 2s 14ms/step - loss: 0.0402 - accuracy: 0.9883 - val_loss: 0.3096 - val_accuracy: 0.9061
Epoch 5/10
131/131 [==============================] - 2s 14ms/step - loss: 0.0172 - accuracy: 0.9959 - val_loss: 0.4017 - val_accuracy: 0.9085
Epoch 6/10
131/131 [==============================] - 2s 16ms/step - loss: 0.0043 - accuracy: 0.9993 - val_loss: 0.5232 - val_accuracy: 0.9002
Epoch 7/10
131/131 [==============================] - 2s 13ms/step - loss: 0.0029 - accuracy: 0.9995 - val_loss: 0.5790 - val_accuracy: 0.902

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[1077,  104],
       [  99,  775]])

In [ ]:
print(accuracy_score(y_test,y_pred))

0.9012165450121654
